# LSTM: Predikere virkeligheten av katastrofetweets

Ressurser:
- https://github.com/bentrevett/pytorch-sentiment-analysis
- https://github.com/thanhtut/info284_lab/tree/master/lab8



Konklusjon:
- Vi oppnår nøyaktighet rundt 80% til tross for lite og rotete data.
<br><br>

Neste gang: 
- inkluder kilder nederst i celle.
- lage program for justering av hyperparamtere.
- loggføre endringer i hyperparametere og nøyaktighet tilknyttet ulike hyperparametere.

In [ ]:
import torch
from torchtext import data
from sklearn.model_selection import train_test_split
import pandas as pd
import random

In [ ]:
#MOUNT DRIVE
#from google.colab import drive
#drive.mount('/content/drive')
#CONTENT_DRIVE = './content/drive'

<h1> Datahåndtering </h1> 
<p> I denne tekstboksen presenteres hva vi gjorde med tekstdataen. </p>
<br>
<h2> Hva: </h2>
<ul> 
<li>Endre smilies til placeholders</li>
<li>Fra caps til småbokstaver, merkes med &ltcaps&gt </li>
<li>Hashtags endres til &lthashtag&gt </li>
<li>Forlengede og gjentatte ord endres og blir tildelt placeholders</li>
<li>Tall, brukernavn og URLer får placeholder </li>
<li>Mellomrom mellom !, ?, - eller = og tekst </li>
</ul> 
<br>
<p>Notat: 
<br> - vi har problemer med encoding. Vi valgte å ikke fokusere på dette da vi er usikre på om det vil være fruktbart for fremtidige analyser.<br> - datahåndteringen vår hadde liten effekt på nøyaktighet. Vi tror dette kan skyldes mye uren data og for lite data.</p>
<br>
<br>


Se: 
- https://docs.python.org/3/library/re.html (regex API)
- https://gist.github.com/tokestermw/cb87a97113da12acb388 (datahåndtering, upolert)
- https://gist.github.com/elyase/839bf4354e442a691d04e48d0df402dd (datahåndtering)
- https://www.freeformatter.com/html-escape.html (Problemer med formateringer?)

In [ ]:
PATH = "/content/drive/My Drive/data-tekstanalyse/"
train = 'df_train_split.csv'
test = 'df_test_split.csv'

df = pd.read_csv(PATH+train)

import regex as re
FLAGS = re.MULTILINE | re.DOTALL

def hashtag(text):
  text = text.group()
  hashtag_body = text[1:]
  if hashtag_body.isupper():
      result = "<hashtag> {} <allcaps".format(hashtag_body.lower())
  else:
      result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
  return result


def allcaps(text):
    text = text.group()
    return text.lower() + " <allcaps> "
 
#def format_fixer(text):
""" is replaced with &quot;
& is replaced with &amp;
< is replaced with &lt;
> is replaced with &gt;""""
#
#  def re_sub(pattern, repl):
#    return re.sub(pattern, repl, text, flags=FLAGS)
#
#  text = re_sub(r"&amp;",r"and")
#  text = re_sub(r"&quot;",r"\"")
#  text = re_sub(r"&lt;",r"<")
#  text = re_sub(r"&gt;",r">")
#  return text

def tokenize(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"/"," / ") #Hva gjør denne?
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.=-]){2,}", r"\1 <repeat>")
    text = re_sub(r"([!?.]){1,}", r" \1 ")
    #text = re_sub(r"[?]",r" ? ")
    #text = re_sub(r"[!]"," ! ")
    #text = re_sub(r"ûª","'")
    #text = re_sub(r"ûï","")
    #text = re_sub(r"û_","")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
  
    #text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    text = re_sub(r"([A-Z]){2,}", allcaps)
    #text = format_fixer(text)
    return text.lower()
    

#import unicodedata


def clean2(dataframe):
  clean_list = []
  for index, row in df.iterrows():
    #try:
     # clean_list.append((tokenize(row.text).lower(),row.target))
    #except:
     # clean_list.append((row.text.lower(),row.target))
     #row.text = row.text.decode('utf-8')
     #row.text = unicodedata.normalize('NFKD', row.text).encode('ascii','ignore')
     #row.text.decode("utf-8").encode("windows-1252").decode("utf-8")

     clean_list.append((tokenize(row.text),row.target))
  return clean_list

    
def clean_frame(dataframe, set_test_size = 0.1):
    clean_list = clean2(dataframe)
    clean_dataset = pd.DataFrame(clean_list, columns = ["text","target"])
    
    #split dataset into training and testing, test is 10% of dataset
    #data_train, data_test = train_test_split(clean_dataset, test_size = set_test_size)
    #return data_train, data_test
    return clean_dataset

data_re = clean_frame(df)

In [ ]:
data_re

,text,target
0,even then our words slip and souls coincide fi...,0
1,had a minute alone with my crush . <rep...,0
2,look: i have collapsed <hashtag> after attem...,0
3,<url> suicide bomber kills <number> in saudi s...,1
4,you call them weekends . i call them bl...,0
...,...,...
260,i'm my own woman crush ? <repeat>,0
261,i keep it out down drown their insults out wit...,0
262,we just happened to get on the same road right...,0
263,young children among those rescued from capsiz...,1


In [ ]:
#Read original data
#PATH_train = "/content/drive/My Drive/data-tekstanalyse/train.csv"
#PATH_test = "/content/drive/My Drive/data-tekstanalyse/test.csv"

#df_train = pd.read_csv(PATH_train)
#df_test = pd.read_csv(PATH_test)

In [ ]:
#make two csv-files
df_train_split, df_test_split = train_test_split(data_re, test_size = 0.2, random_state = 0)


df_test_split.to_csv('/content/drive/My Drive/data-tekstanalyse/df_test_split.csv',index = False)
df_train_split.to_csv('/content/drive/My Drive/data-tekstanalyse/df_train_split.csv', index = False)




In [ ]:
##MAKE ITERATOR FOR CLEAN DATA

#Make fields
#TEXT = data.Field(tokenize = 'spacy', include_lengths = True)

TEXT = data.Field(include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

PATH = "/content/drive/My Drive/data-tekstanalyse/"
fields = [('text', TEXT),('target', LABEL)]

df = pd.read_csv('/content/drive/My Drive/data-tekstanalyse/df_test_split.csv')

#Load data
train_data, test_data = data.TabularDataset.splits(
                                        path = PATH,
                                        train = 'df_train_split.csv',
                                        test = 'df_test_split.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

#Split data
SEED = 1234
train_data, valid_data = train_data.split(random_state = random.seed(SEED), split_ratio=0.80)

In [ ]:
print(vars(train_data.examples[0]))
print(vars(valid_data.examples[0]))
print(vars(test_data.examples[0]))

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

{'text': ['<hashtag>', 'worldnews', 'fears', 'over', 'missing', 'migrants', 'in', 'med', '-', 'bbc', '<allcaps>', 'news', '-', 'home:', 'rescuers', 'search', 'for', 'survivors', 'after', 'a', 'boat', 'carr', '.', '<repeat>', '<url>'], 'target': '1'}
{'text': ['petition', '|', 'heartless', 'owner', 'that', 'whipped', 'horse', 'until', 'it', 'collapsed', 'is', 'told', 'he', 'can', 'keep', '<allcaps>', 'his', 'animal', '!', 'act', 'now', '!', '<url>'], 'target': '0'}
{'text': ['achievement', 'unlocke<smile>', 'replaced', 'light', 'socket;', 'did', 'not', 'electrocute', 'self'], 'target': '0'}
Number of training examples: 170
Number of validation examples: 42
Number of testing examples: 53


<h1> Hyperparametere </h1>

<ul> 
  <li><strong>MAX_VOCAB_SIZE:</strong> Hvor mange ord skal nettverket lære seg, merker ukjente ord med &lt;unk&gt;. </li>
  <li><strong>BATCH_SIZE:</strong> Hvor mange treningseksempler benyttes i hver iterasjon. </li>
  <br>

  <li><strong>INPUT_DIM</strong> = len(TEXT.vocab): Størrelse på input dimensjon (hvor mange noder). </li>
  <li><strong>EMBEDDING_DIM:</strong> Dimensjon på embedding layer. </li>
  <li><strong>OUTPUT_DIM:</strong> Dimensjon på output. </li>
  <br>

  <li><strong>PAD_IDX</strong> = TEXT.vocab.stoi[TEXT.pad_token]; dette er hvordan padding blir merket: &lt; pad &gt; </li>
  <li><strong>UNK_IDX:</strong> Hvordan unk blir merket: &lt;unk&gt;  </li>
  <li><strong>pretrained_embeddings:</strong> embeddings hentet gjennom glove </li>
  <br>

  <li><strong>N_LAYERS:</strong> Antall lag i LSTM. </li>
  <li><strong>BIDIRECTIONAL:</strong> Se lengre ned i dokumentet. </li>
  <li><strong>DROPOUT:</strong> Kan anses som grad av regualisering (motsatte av overfitting; når modellen blir for tilpasset treningsdataen slik at det går utover nøyaktigheten på testdataen). Høyere dropout rate fører til mer regualisering. Regualiseringen gjøres gjennom å forstyrre tilfeldige nøyroner i nettverket. </li>
  
  <br>
  <li><strong>optimizer:</strong> Hvilken optimeringsalgoritme som skal benyttes. Nøyrale nettverk har som mål redusere antall feilklassifiseringer. Dette gjøres ved å beregne gradienter. Gradientene blir beregnet med ulike algoritmer. De vanligste algoritmene er Adam og SDG. Se https://pytorch.org/docs/stable/optim.html for tilgjengelig alogritmer. </li>
  <li><strong>Learning rate (lr):</strong> Denne parameteren angir hvor lange "steg" optimeringsalgoritmen skal ta i hver iterasjon. Lange steg gir en raskere læringsprosess men kan føre til at en bommer på "målet" (målet vil være et globalt minimum for kostnadsfunksjonen). Korte steg gir en tregere læringsprosess, men kan føre til at en kommer nærmere globalet minimum. </li>
  <li><strong>criterion:</strong> angir hvordan loss beregnes, det kan anses som et mål på avstand mellom predikert svar og riktig svar. Vi bruker BCEWithLogitLoss, se https://pytorch.org/docs/master/generated/torch.nn.BCEWithLogitsLoss.html og https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79. </li>
  <li><strong>nr of epochs:</strong> Antall iterasjoner som brukes i læringsprosessen.</li>
</ul>

**Max vocab size**



*   MAX_VOCAB_SIZE setter vi et tak på hvor mange unike ord (eller tegn) glove skal ta med i maskinlæringa. Ordene blir rangert etter hvor mange ganger det forekommer. I dette tilfellet vil de 10.000 mest brukte ordene bli tatt med. Hvor mange ord man bør ta med vil avhenge av hvor mange unike ord som finnes i datasettet. 
*   Det er viktig å merke seg at unike ord blir identifisert i vid forstand. F.eks. regnes '?' og '**' som unike ord. Dette gjelder også varianter av samme ord slik som 'er' og 'var', 'ku' og 'kyr' eller 'ski' og 'skiene'. (Merk at glove er trent på engelske ord). Samtidig vil også 'hei!' og 'hei' kunne gjennkjennes som egne ord med mindre man har tatt høyde for dette i data rensinga.
* Avhengig av hva slags data man har kan det være tilfeller der man ønsker å beholde slike variasjoner eller ensrette slike variasjoner. Dette gjøres i data rensingen.
* Det antallet vi har her med 10_000 er ganske stort med tanke på dataene vi bruker. Dette vil ha stor innvirkningen på treffsikkerheten i nettverket da man tar med lite brukte ord, og det kan ofte være lurt å fokusere på et mindre antall unike ord.



<strong>Embeddings </strong> <br>
<ul>
  <li>RNN med LSTM har et lag kalt embedding layer. Dette laget ligger mellom input-laget (hvor ord blir sendt inn) og LSTM-laget. Input-laget mottar informasjon som one-hot vektorer, og sender dem videre til embedding-laget. 
  </li>
 <li>La oss si at du har laget en ordbok med 4 ord: "jeg", "liker", "is", "elsker". Med one-hot vektorer vil "jeg" og "liker" være like nærme hverandre som "liker" og "elsker". For mennekser er det lett å se at "liker" og "elsker" har semantiske likhetstrekk. I embedding-laget er ikke dette lengre tilfellet. Ord med lik sematisk betydning nærme hverandre i tensoren (embedding-laget blir representert i en fledimensjonal vektor (tensor)). (https://www.quora.com/What-is-the-embedding-layer-in-LSTM-long-short-term-memory)
 </li>
 <li>Vi benytter oss av en forhåndslært tensor som er konstruert av alogritmen glove (https://nlp.stanford.edu/projects/glove/). Denne er trent på engelske ord. </li>
 <li>Ved hjelp av glove unngår vi å starte fra "scratch". Typisk vil et nøyralt nettverk ha et tilfeldig utgangspunkt. Med glove tensoren slipper vi dette, og læringsprosessen er mye raskere. </li>
</ul>

In [ ]:
#VOCAB FOR DIRTY DATA
MAX_VOCAB_SIZE = 10_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_) #initialiserer <unk> i embedding til tall hentet fra normalfordeling

LABEL.build_vocab(train_data)

In [ ]:
#ITERATOR FOR CLEAN DATA
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

<h1>Layout of network</h1>
<h2>Bidirectional long short-term memory feedback network for tekstanalyse.</h2>

<strong>LSTM (long short-term memory)</strong> er en måte å behandle inputene i en RNN på. Den vil i de fleste tilfeller være bedre å bruke og er samtidig beheftet med færre problemer, slik som vanishing gradient, samenlignet med standard RNN. For å forstå hva LSTM gjør er det viktig å huske på hva en RNN gjør. RNN går kort fortalt ut på at modellen leser inn mindre stykker av f.eks. en lengre tekst og samtidig bevarer det den har lest før. Med andre ord behandler modellen en ny input som en funksjon av gjeldende input pluss en 'state' som representerer output fra foregående steg. Dette er en forenkling av hvordan vi mennesker prosesserer ny informasjon, f.eks. leser en tekst: Vi leser ordene i hvert øyekast (input) og vurderer dette opp mot de foregående ordene (state). Det har vist seg at selv om state teoretisk skal kunne ta med seg informasjon fra 'langt tilbake' og lære av dette, er det vanskelig i praksis, blant annet fordi relativt små signaler fra langt tilbake har en tendens til å viskes ut. LSTM løser dette ved å legge til en egen prosess 'carry track', som tar med seg informasjon fra foregående steg på i en separat prosess. State er i LSTM et resultat av både foregående output, men også rå inputen fra forrige steg. Samtidig er behandlingen av hver ny input i LSTM RNN et resultat av ikke bare nåværende input og state (som i standard RNN), men også carry fra alle foregående steg.

<strong>Bidirectional</strong> innebærer at nodene i et lag av nettverket kan bruke informasjon fra tidligere òg framtidige input i modellen, altså trekke på nabonodene i samme 'linje'. Dette er særlig nyttig i tilfeller der konteksten er viktig. Et eksempel er i oversettelse der betydningen av et ord kan kjenngjennes bedre hvis modellen får se på ordet før og etter et ord. Bidirectional er en type RNN (recurrent neural network), og sammen med LSTM er særlig egnet til å arbeide med tekst og språk, f.eks. tale-til-tekst og kjengjenning av håndskrift.

Vi prøvde å ha to LSTM etter hverandre. Det forbedredet ikke nøyaktigheten til modellen. 

Se:
- https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/
- https://en.wikipedia.org/wiki/Long_short-term_memory
- https://colah.github.io/posts/2015-08-Understanding-LSTMs/
- https://arxiv.org/pdf/1801.01078.pdf (om RNN dens utvikling)

In [ ]:
#lay-out of RNN
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx, n_layers2 = None, dropout2 = None):
        
        super().__init__()
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        #LSTM-layers
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        #self.rnn2 = nn.LSTM(hidden_dim, 
        #                   hidden_dim, 
        #                   num_layers=n_layers2,
        #                    bidirectional=bidirectional, 
        #                   dropout=dropout2)
        
        #fully connected
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [ ]:
#Hyperparameters for clean data
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 5
BIDIRECTIONAL = True
DROPOUT = 0.4
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
#N_LAYERS2 = 3
#DROPOUT_2 = 0.2


#pretrained embeddings
pretrained_embeddings = TEXT.vocab.vectors

#index for unkown words
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

#model with hyperparam
model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX,
            )

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)
print(pretrained_embeddings.shape)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.8438,  1.4076, -0.5572,  ...,  0.3167,  0.0515,  0.6809],
        ...,
        [-1.0433,  0.1767, -1.1038,  ...,  0.3532,  0.7137,  0.2933],
        [ 0.7033, -0.1868,  2.2958,  ...,  2.3021, -0.5277,  2.0576],
        [ 0.2183,  1.0443, -1.1184,  ..., -0.3706,  0.3375,  0.4192]])
torch.Size([1252, 100])


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,166,737 trainable parameters


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-2.1744, -0.2356, -0.6259,  ..., -0.6309,  0.1220, -0.5204],
        [ 1.9446, -0.4448, -1.2926,  ...,  0.9853,  0.2777,  1.1442],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 1.2011,  0.5408,  0.2319,  ...,  1.0922, -0.1467,  0.8313],
        [ 1.8574,  0.1586, -0.9055,  ..., -0.8275,  0.8096,  0.1784],
        [-1.1062,  0.3486,  0.9329,  ..., -1.5898, -1.1218, -0.4589]])

In [ ]:
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 1.2011,  0.5408,  0.2319,  ...,  1.0922, -0.1467,  0.8313],
        [ 1.8574,  0.1586, -0.9055,  ..., -0.8275,  0.8096,  0.1784],
        [-1.1062,  0.3486,  0.9329,  ..., -1.5898, -1.1218, -0.4589]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr = 0.0009)

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.target)
        
        acc = binary_accuracy(predictions, batch.target)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.target)
            
            acc = binary_accuracy(predictions, batch.target)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 6

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.692 | Train Acc: 51.79%
	 Val. Loss: 0.698 |  Val. Acc: 52.38%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.675 | Train Acc: 56.03%
	 Val. Loss: 0.698 |  Val. Acc: 52.38%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.623 | Train Acc: 58.41%
	 Val. Loss: 0.671 |  Val. Acc: 52.38%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.629 | Train Acc: 67.58%
	 Val. Loss: 0.650 |  Val. Acc: 57.14%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.621 | Train Acc: 72.74%
	 Val. Loss: 0.607 |  Val. Acc: 78.57%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.518 | Train Acc: 76.17%
	 Val. Loss: 0.598 |  Val. Acc: 71.43%


In [ ]:
#model.load_state_dict(torch.load('tekstanalyse.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.607 | Test Acc: 64.15%


In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
i = 0
for text in data_re['text']:
  predicted_label = 0
  pred = predict_sentiment(model,text)
  if pred < 0.5: predicted_label = 0
  else: predicted_label = 1
  i += 1 
  print(f'Sentence: {str(text)} | Prediction score: {round(pred,2)} | Predicted label: {predicted_label}')

  if i == 20: break

Sentence: even then our words slip and souls coincide finer than subatomic spells just as we collide <url> | Prediction score: 0.21 | Predicted label: 0
Sentence: had a minute alone with my crush    .     <repeat>it was an overrated experience    .     <repeat>smh | Prediction score: 0.41 | Predicted label: 0
Sentence: look:  i  have collapsed <hashtag> after attempting to munch an endangered species    .     | Prediction score: 0.47 | Predicted label: 0
Sentence: <url> suicide bomber kills <number> in saudi security site mosque - reuters <url> | Prediction score: 0.88 | Predicted label: 1
Sentence: you call them weekends    .     i call them bloody mary times    .     this summer's been full of them    .     my new <url> | Prediction score: 0.27 | Predicted label: 0
Sentence: san antonio stars head coach dan hughes was just carted to the locker room after one of his guards collided with    .     <repeat> <url> | Prediction score: 0.74 | Predicted label: 1
Sentence: big data and social